In [78]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import re


In [3]:
DATA_DIR = "data"
DAILY_DIR = os.path.join(DATA_DIR, "DAILYLINES")

player_props = os.listdir(DAILY_DIR)
player_props = [os.path.join(DAILY_DIR, f) for f in player_props if f.endswith(".html")]

In [79]:

def parse_html(pp):
    with open(pp, encoding=None, errors="replace") as f:
        html = f.read()

    soup = BeautifulSoup(html, features="html.parser")

    return soup


In [63]:

markets = {
    'pontos mais/menos': 'PTS',
    'rebotes mais/menos': 'TRB',
    'assistências mais/menos': 'AST',
    'total arremessos de três pontos marcados +/-': '3P',
    'roubos mais/menos': 'STL',
    'bloqueios mais/menos': 'BLK',
    'turnover mais/menos': 'TOV',
    'pontos + rebotes + assistências mais/menos': 'PRA',
    'pontos + rebotes mais/menos': 'PR',
    'pontos + assistências mais/menos': 'PA'
}

teams_extended = {
    'boston celtics': 'BOS', 'brooklyn nets': 'BKN', 'new york knicks': 'NYK', 'philadelphia 76ers': 'PHI',
    'toronto raptors': 'TOR',
    'chicago bulls': 'CHI', 'cleveland cavaliers': 'CLE', 'detroit pistons': 'DET', 'indiana pacers': 'IND',
    'milwaukee bucks': 'MIL',
    'golden state warriors': 'GSW', 'los angeles clippers': 'LAC', 'los angeles lakers': 'LAL', 'phoenix suns': 'PHX',
    'sacramento kings': 'SAC',
    'atlanta hawks': 'ATL', 'charlotte hornets': 'CHA', 'miami heat': 'MIA', 'orlando magic': 'ORL',
    'washington wizards': 'WAS',
    'denver nuggets': 'DEN', 'minnesota timberwolves': 'MIN', 'oklahoma city thunder': 'OKC',
    'portland trail blazers': 'POR', 'utah jazz': 'UTA',
    'dallas mavericks': 'DAL', 'houston rockets': 'HOU', 'memphis grizzlies': 'MEM', 'new orleans pelicans': 'NOP',
    'san antonio spurs': 'SAS'
}

In [60]:
def find_teams(archive):
    teams = []
    new = archive.split('\\')[-1]
    new = new.replace('-', ' ')
    new = new.replace('.html', '')
    new = new.split()
    for n in new:
        try:
            teams.append(teamsDic[n])
        except:
            continue
    return teams


def parse_game(player_props):
    for pp in player_props:
        soup = parse_html(pp)

        lines = soup.get_text(separator="--!--")
        lines_list = lines.split("--!--")
        for i, value in enumerate(lines_list):
            try:
                lines_list[i] = float(value)
            except ValueError:
                pass

        strings_to_remove = ['', 'linha', 'mais de', 'menos de']
        remove_index = []
        for j in range(len(lines_list)):
            lines_list[j] = lines_list[j].lower().strip() if isinstance(lines_list[j], str) else lines_list[j]
            if lines_list[j] in strings_to_remove:
                remove_index.append(j)
        remove_index.reverse()
        [lines_list.pop(i) for i in remove_index]
        create_csv(pp, lines_list)



In [107]:
def create_csv(pp, lines):
    game_html = pp.split('\\')[-1]
    game = game_html.split('.')[0]
    file_path = DAILY_DIR+"\\"+game+"_props.csv"
    headers_row = ["player", "team", "market", "line", "over", "under"]
    df = pd.DataFrame(columns=headers_row)

    player = ""
    team = ""
    market = ""
    line = 0
    over = 0
    under = 0

    ending_points = ['double double', "pontos + bloqueios mais/menos", "roubadas + bloqueios mais/menos", "rebotes + assistências mais/menos"]
    while len(lines) != 0:
        actual = lines.pop(0)

        if actual in ending_points:
            print("Fim de Scripts")
            break
        elif actual == 'ver menos':
            actual = lines.pop(0)
        elif actual in markets.keys():
            market = actual
        elif actual in teams_extended.keys():
            team = teams_extended[actual]
        else:
            player = actual
            line = float(lines.pop(0))
            over = float(lines.pop(0))
            under = float(lines.pop(0))
            new_row = pd.DataFrame(
                {"player": player, "team": team, "market": market, "line": line, "over": over, "under": under}, index=[0])
            df = pd.concat([new_row,df.loc[:]]).reset_index(drop=True)

    print('---------------------------------------------')
    unique_teams = df['team'].unique()

    def update_opp(row):
        opposing_team = [team for team in unique_teams if team != row['team']]
        return opposing_team[0] if opposing_team else ''

    df['opp'] = df.apply(update_opp, axis=1)

    df.to_csv(file_path)

In [108]:
parse_game(player_props)


Fim de Scripts
---------------------------------------------
Fim de Scripts
---------------------------------------------
Fim de Scripts
---------------------------------------------
Fim de Scripts
---------------------------------------------
Fim de Scripts
---------------------------------------------
